<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Computação em Nuvem II
Caderno de **Exercícios**<br>
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **Tópicos**

<ol type="1">
  <li>AWS Lambda;</li>
  <li>AWS Step Functions;</li>
  <li>AWS EventBridge.</li>
</ol>

---

# **Exercícios**

## 1\. AWS Lambda

### Código


```sql
import json
import logging
from datetime import datetime

import boto3
import urllib3
from botocore.exceptions import ClientError


def lambda_handler(event, context) -> bool:

  # -- setup

  URL = 'https://api.bcb.gov.br/dados/serie/bcdata.sgs.4392/dados?formato=json'
  BRONZE_BUCKET = 'modulo38-bronze'

  client = boto3.client('s3')

  date = datetime.now().strftime('%Y-%m-%d')
  filename_json = f'stock-exchange-{date}.json'

  # -- extract

  try:
    http = urllib3.PoolManager()
    response = http.request(url=URL, method='get')
  except Exception as exc:
    raise exc
  else:
    data = json.loads(response.data.decode())
    logging.info(msg=data)

  # -- transform

  ...

  # -- load

  try:
      with open(f'/tmp/{filename_json}', mode='w', encoding='utf8') as fp:
          json.dump(data, fp)
      client.upload_file(Filename=f'/tmp/{filename_json}', Bucket=BRONZE_BUCKET, Key=filename_json)
  except ClientError as exc:
      raise exc

  return json.dumps(dict(status=True))
  ```

### Prints

Configuração realizada no IAM:

![configuração-IAM-bronze](https://github.com/mariaefoliveira/Analise-de-Dados/blob/main/Atividades%20-%20Ebac/Atividade%2038%20-%20Ebac%20-%20Nuvem/Imagens/configuracao-lambda-silver-bronze.png?raw=true)


Arquivo criado no S3:

![arquivo-S3-bronze](https://github.com/mariaefoliveira/Analise-de-Dados/blob/main/Atividades%20-%20Ebac/Atividade%2038%20-%20Ebac%20-%20Nuvem/Imagens/arquivo-lambda-bronze.png?raw=true)

### Código

```sql
import json
from datetime import datetime

import boto3
from botocore.exceptions import ClientError


def lambda_handler(event, context) -> bool:
    # -- setup
    BRONZE_BUCKET = 'modulo38-bronze'
    SILVER_BUCKET = 'modulo38-silver'
    
    client = boto3.client('s3')
    
    date = datetime.now().strftime('%Y-%m-%d')
    filename_csv = f'stock-exchange-{date}.csv'
    filename_json = f'stock-exchange-{date}.json'
    
    # -- extract
    client.download_file(BRONZE_BUCKET, filename_json, f'/tmp/{filename_json}')
    
    with open(f"/tmp/{filename_json}", mode='r', encoding='utf8') as fp:
        data = json.load(fp)
    
    # -- transform
    transformed_data = []
    
    for record in data:
        transformed_record = {
            'taxa': record['valor'].replace(',', '.'),
            'dataTaxa': datetime.strptime(record['data'], '%d/%m/%Y').strftime('%Y-%m-%d'),
        }
        transformed_data.append(transformed_record)
    
    # -- load
    try:
        with open(f'/tmp/{filename_csv}', mode='w', encoding='utf8') as fp:
            fp.write('taxa,dataTaxa\n')
            for record in transformed_data:
                fp.write(f"{record['taxa']},{record['dataTaxa']}\n")
        client.upload_file(Filename=f'/tmp/{filename_csv}', Bucket=SILVER_BUCKET, Key=f'data_referencia={date}/{filename_csv}')
    except ClientError as exc:
        raise exc
    
    return json.dumps(dict(status=True))
```

### Prints

Configuração realizada no IAM:

![configuração-IAM-silver](https://github.com/mariaefoliveira/Analise-de-Dados/blob/main/Atividades%20-%20Ebac/Atividade%2038%20-%20Ebac%20-%20Nuvem/Imagens/configuracao-lambda-silver-bronze.png?raw=true)


Arquivo criado no S3:

![arquivo-S3-silver](https://github.com/mariaefoliveira/Analise-de-Dados/blob/main/Atividades%20-%20Ebac/Atividade%2038%20-%20Ebac%20-%20Nuvem/Imagens/arquivo-lambda-silver.png?raw=true)

```sql

import json
from datetime import datetime

import boto3
from botocore.exceptions import ClientError


def lambda_handler(event, context) -> bool:

  # -- setup

  SILVER_BUCKET = 'modulo38-silver'

  query = f"""
  CREATE EXTERNAL TABLE IF NOT EXISTS cdi (
    taxa double,
    dataTaxa string
  )
  PARTITIONED BY (
    data_referencia string
  )
  ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
  WITH SERDEPROPERTIES ('separatorChar'=',')
  LOCATION 's3://{SILVER_BUCKET}/'
  """

  client = boto3.client('athena')

  # -- create

  try:
    client.start_query_execution(
      QueryString=query,
      ResultConfiguration={'OutputLocation': 's3://modulo-37-resultados-query/'}
    )
  except ClientError as exc:
    raise exc

  # -- update

  try:
    client.start_query_execution(
      QueryString='MSCK REPAIR TABLE cdi',
      ResultConfiguration={'OutputLocation': 's3://modulo-37-resultados-query/'}
    )
  except ClientError as exc:
    raise exc

  return json.dumps(dict(status=True))
```



Configuração realizada no IAM:

![configuração-IAM-tabela](https://github.com/mariaefoliveira/Analise-de-Dados/blob/main/Atividades%20-%20Ebac/Atividade%2038%20-%20Ebac%20-%20Nuvem/Imagens/configuracao-lambda-tabela.png?raw=true)

Athena-cdi:

![tabela-athena-cdi](https://github.com/mariaefoliveira/Analise-de-Dados/blob/main/Atividades%20-%20Ebac/Atividade%2038%20-%20Ebac%20-%20Nuvem/Imagens/tabela-cdi.png?raw=true)

![sql](https://github.com/mariaefoliveira/Analise-de-Dados/blob/main/Atividades%20-%20Ebac/Atividade%2038%20-%20Ebac%20-%20Nuvem/Imagens/sql-tabela.png?raw=true)

## 2\. AWS Step Functions

![step-function-configurado](https://github.com/mariaefoliveira/Analise-de-Dados/blob/main/Atividades%20-%20Ebac/Atividade%2038%20-%20Ebac%20-%20Nuvem/Imagens/grafico-step-functions.png?raw=true)

![step-function-ok](https://github.com/mariaefoliveira/Analise-de-Dados/blob/main/Atividades%20-%20Ebac/Atividade%2038%20-%20Ebac%20-%20Nuvem/Imagens/grafico-step-functions-ok.png?raw=true)

## 3\. AWS EventBridge

![Cronograma](https://github.com/mariaefoliveira/Analise-de-Dados/blob/main/Atividades%20-%20Ebac/Atividade%2038%20-%20Ebac%20-%20Nuvem/Imagens/cronograma-criado.png?raw=true)